<a href="https://colab.research.google.com/github/Ebrahim-Ghaith/Research/blob/main/hypered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch

class Net_m_task_CNN(nn.Module):
    def __init__(self, n_in, n_hidden, n_out_Reg, n_out_clas, p_dropout, U, Ass, out_channel, kernel_s, padding):
        super(Net_m_task_CNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=out_channel, kernel_size=kernel_s, stride=1, padding=padding)
        self.relu1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm2d(num_features=out_channel)
        self.do1 = nn.Dropout2d(p_dropout)

        self.cnn2 = nn.Conv2d(in_channels=out_channel, out_channels=out_channel, kernel_size=kernel_s, stride=1, padding=padding)
        self.relu2 = nn.LeakyReLU()
        self.bn2 = nn.BatchNorm2d(num_features=out_channel)
        self.do2 = nn.Dropout2d(p_dropout)

        self.cnn3 = nn.Conv2d(in_channels=out_channel, out_channels=8, kernel_size=kernel_s, stride=1, padding=padding)
        self.relu3 = nn.LeakyReLU()
        self.bn3 = nn.BatchNorm2d(num_features=8)
        self.do3 = nn.Dropout2d(p_dropout)

        # Fully connected 1 (readout)
        x_new = (U + 2 * padding - kernel_s) + 1
        y_new = (Ass + 2 * padding - kernel_s) + 1

        nn_in_fc = 8 * (x_new + 2 * padding - kernel_s + 1) * (y_new + 2 * padding - kernel_s + 1)

        self.fc20 = nn.Linear(nn_in_fc, n_hidden)
        self.bn20 = nn.BatchNorm1d(n_hidden)
        self.relu20 = nn.LeakyReLU()
        self.do20 = nn.Dropout(p_dropout)

        self.fc30 = nn.Linear(n_hidden, n_hidden)
        self.bn30 = nn.BatchNorm1d(n_hidden)
        self.relu30 = nn.LeakyReLU()
        self.do30 = nn.Dropout(p_dropout)

        # Linear function (readout)
        self.fc7R = nn.Linear(n_hidden, n_out_Reg)

        # Linear function (readout)
        self.fc8R = nn.Linear(n_hidden, n_out_Reg)

        # Linear function (readout)
        self.fc9C = nn.Linear(n_hidden, n_out_clas)

    def forward(self, x):  # always

        out = self.cnn1(x)
        out = self.do1(out)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.cnn2(out)
        out = self.do2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.cnn3(out)
        out = self.do3(out)
        out = self.bn3(out)
        out = self.relu3(out)

        out = out.view(out.size(0), -1)

        out = self.fc20(out)
        out = self.do20(out)
        out = self.relu20(out)
        out = self.bn20(out)

        out = self.fc30(out)
        out = self.do30(out)
        out = self.relu30(out)
        out = self.bn30(out)

        # Linear function (readout)  ****** LINEAR ******
        outR1 = self.fc7R(out)

        # Linear function (readout)  ****** LINEAR ******
        outR2 = self.fc8R(out)

        # Linear function (readout)  ****** LINEAR ******
        outC = self.fc9C(out)

        return outR1, outR2, outC


In [2]:
# from networks_arch import Net_m_task_CNN
import torch.nn as nn
import torch

class Networks_activations(object):
    def __init__(self,
                 Us,
                 Mr,
                 Nrf,
                 K,
                 K_limited,
                 Noise_pwr,
                 device,
                 device_ids,
                 n_input,
                 n_hidden,
                 n_output_reg,
                 n_output_clas,
                 p_dropout,
                 out_channel,
                 kernel_s,
                 padding
                 ):
        self.Us = Us
        self.Mr = Mr
        self.Nrf = Nrf
        self.K = K
        self.K_limited = K_limited
        self.Noise_pwr = Noise_pwr
        self.device = device
        self.dev_id = device_ids
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output_reg = n_output_reg
        self.n_output_clas = n_output_clas
        self.p_dropout = p_dropout
        self.out_channel = out_channel
        self.kernel_s = kernel_s
        self.padding = padding

    def Network_m_Task(self):
        if self.device.type == 'cuda':
            return nn.DataParallel(Net_m_task_CNN(self.n_input, self.n_hidden, self.n_output_reg, self.n_output_clas, self.p_dropout, self.Us, self.K_limited, self.out_channel, self.kernel_s, self.padding), device_ids=self.dev_id).to(self.device)
        else:
            return Net_m_task_CNN(self.n_input, self.n_hidden, self.n_output_reg, self.n_output_clas,
                self.p_dropout, self.Us, self.K_limited, self.out_channel, self.kernel_s, self.padding)

    def Inp_MT(self, RSSI):
        Inputs_MT = RSSI.reshape(len(RSSI), 1, self.Us, self.K)[:, :, :, 0:self.K_limited].float().to(self.device)
        return Inputs_MT


In [3]:
"""
@author: Hamed Hojatian

Complex aljebra for pytorch in case that pytorch
version doesn't support complex tensor (V 1.8 and
later support complex tensor and aljebra)
"""


import torch
from numpy import genfromtxt
import numpy as np

def Th_comp_matmul(Ar, Ai, Br, Bi):  # Complex matmul pytorch function ########
    if Ar.ndim == 3 and Br.ndim == 3:
        a_th = torch.cat((torch.cat((Ar, -Ai), dim=2), torch.cat((Ai, Ar), dim=2)), dim=1)
        b_th = torch.cat((torch.cat((Br, -Bi), dim=2), torch.cat((Bi, Br), dim=2)), dim=1)
        c_th = torch.matmul(a_th, b_th)
        c_th_r = c_th[:, 0:int(c_th.shape[1] / 2), 0:int(c_th.shape[2] / 2)]
        c_th_i = c_th[:, int(c_th.shape[1] / 2):, 0:int(c_th.shape[2] / 2)]
    elif Ar.ndim == 2 and Br.ndim == 2:
        a_th = torch.cat((torch.cat((Ar, -Ai), dim=1), torch.cat((Ai, Ar), dim=1)), dim=0)
        b_th = torch.cat((torch.cat((Br, -Bi), dim=1), torch.cat((Bi, Br), dim=1)), dim=0)
        c_th = torch.matmul(a_th, b_th)
        c_th_r = c_th[0:int(c_th.shape[0] / 2), 0:int(c_th.shape[1] / 2)]
        c_th_i = c_th[int(c_th.shape[0] / 2):, 0:int(c_th.shape[1] / 2)]
    elif Ar.ndim == 4 and Br.ndim == 4:
        a_th = torch.cat((torch.cat((Ar, -Ai), dim=3), torch.cat((Ai, Ar), dim=3)), dim=2)
        b_th = torch.cat((torch.cat((Br, -Bi), dim=3), torch.cat((Bi, Br), dim=3)), dim=2)
        c_th = torch.matmul(a_th, b_th)
        c_th_r = c_th[:, :, 0:int(c_th.shape[2] / 2), 0:int(c_th.shape[3] / 2)]
        c_th_i = c_th[:, :, int(c_th.shape[2] / 2):, 0:int(c_th.shape[3] / 2)]
    elif Ar.ndim == 5 and Br.ndim == 5:
        a_th = torch.cat((torch.cat((Ar, -Ai), dim=4), torch.cat((Ai, Ar), dim=4)), dim=3)
        b_th = torch.cat((torch.cat((Br, -Bi), dim=4), torch.cat((Bi, Br), dim=4)), dim=3)
        c_th = torch.matmul(a_th, b_th)
        c_th_r = c_th[:, :, :, 0:int(c_th.shape[3] / 2), 0:int(c_th.shape[4] / 2)]
        c_th_i = c_th[:, :, :, int(c_th.shape[3] / 2):, 0:int(c_th.shape[4] / 2)]
    elif Ar.ndim * Br.ndim == 12:
        if Ar.ndim == 4:
            a_th = torch.cat((torch.cat((Ar, -Ai), dim=3), torch.cat((Ai, Ar), dim=3)), dim=2)
            b_th = torch.cat((torch.cat((Br, -Bi), dim=2), torch.cat((Bi, Br), dim=2)), dim=1)
            c_th = torch.matmul(a_th, b_th)
            c_th_r = c_th[:, :, 0:int(c_th.shape[2] / 2), 0:int(c_th.shape[3] / 2)]
            c_th_i = c_th[:, :, int(c_th.shape[2] / 2):, 0:int(c_th.shape[3] / 2)]
        elif Br.ndim == 4:
            a_th = torch.cat((torch.cat((Ar, -Ai), dim=2), torch.cat((Ai, Ar), dim=2)), dim=1)
            b_th = torch.cat((torch.cat((Br, -Bi), dim=3), torch.cat((Bi, Br), dim=3)), dim=2)
            c_th = torch.matmul(a_th, b_th)
            c_th_r = c_th[:, :, 0:int(c_th.shape[2] / 2), 0:int(c_th.shape[3] / 2)]
            c_th_i = c_th[:, :, int(c_th.shape[2] / 2):, 0:int(c_th.shape[3] / 2)]
    elif Ar.ndim * Br.ndim == 20:
        if Ar.ndim == 5:
            a_th = torch.cat((torch.cat((Ar, -Ai), dim=4), torch.cat((Ai, Ar), dim=4)), dim=3)
            b_th = torch.cat((torch.cat((Br, -Bi), dim=3), torch.cat((Bi, Br), dim=3)), dim=2)
            c_th = torch.matmul(a_th, b_th)
            c_th_r = c_th[:, :, :, 0:int(c_th.shape[3] / 2), 0:int(c_th.shape[4] / 2)]
            c_th_i = c_th[:, :, :, int(c_th.shape[3] / 2):, 0:int(c_th.shape[4] / 2)]
        elif Br.ndim == 5:
            a_th = torch.cat((torch.cat((Ar, -Ai), dim=3), torch.cat((Ai, Ar), dim=3)), dim=2)
            b_th = torch.cat((torch.cat((Br, -Bi), dim=4), torch.cat((Bi, Br), dim=4)), dim=3)
            c_th = torch.matmul(a_th, b_th)
            c_th_r = c_th[:, :, :, 0:int(c_th.shape[3] / 2), 0:int(c_th.shape[4] / 2)]
            c_th_i = c_th[:, :, :, int(c_th.shape[3] / 2):, 0:int(c_th.shape[4] / 2)]
    else:
        raise Exception('the dimension is not defined for Th_comp_matmul.')

    return c_th_r, c_th_i

def Th_inv(Ar, Ai):  # Complex inverse pytorch function ########
    Ar_inv = torch.inverse(Ar + torch.matmul(torch.matmul(Ai, torch.inverse(Ar)), Ai))
    Ai_inv = - torch.matmul(torch.matmul(torch.inverse(Ar), Ai), Ar_inv)
    return Ar_inv, Ai_inv

def Th_pinv(Ar, Ai):  # Complex inverse pytorch function ########
    if Ar.ndim == 2:
        if Ar.shape[0] < Ar.shape[1]:
            Tempr, Tempi = Th_comp_matmul(Ar, Ai, Ar.T, -Ai.T)
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar.T, -Ai.T, Ar_inv, Ai_inv)
        elif Ar.shape[0] > Ar.shape[1]:
            Tempr, Tempi = Th_comp_matmul(Ar.T, -Ai.T, Ar, Ai)
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar_inv, Ai_inv, Ar.T, -Ai.T)
        elif Ar.shape[0] == Ar.shape[1]:
            return Th_inv(Ar, Ai)
    elif Ar.ndim == 3:
        if Ar.shape[1] < Ar.shape[2]:
            Tempr, Tempi = Th_comp_matmul(Ar, Ai, Ar.permute(0, 2, 1), -Ai.permute(0, 2, 1))
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar.permute(0, 2, 1), -Ai.permute(0, 2, 1), Ar_inv, Ai_inv)
        elif Ar.shape[1] > Ar.shape[2]:
            Tempr, Tempi = Th_comp_matmul(Ar.permute(0, 2, 1), -Ai.permute(0, 2, 1), Ar, Ai)
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar_inv, Ai_inv, Ar.permute(0, 2, 1), -Ai.permute(0, 2, 1))
        elif Ar.shape[1] == Ar.shape[2]:
            return Th_inv(Ar, Ai)
    elif Ar.ndim == 4:
        if Ar.shape[2] < Ar.shape[3]:
            Tempr, Tempi = Th_comp_matmul(Ar, Ai, Ar.permute(0, 1, 3, 2), -Ai.permute(0, 1, 3, 2))
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar.permute(0, 1, 3, 2), -Ai.permute(0, 1, 3, 2), Ar_inv, Ai_inv)
        elif Ar.shape[2] > Ar.shape[3]:
            Tempr, Tempi = Th_comp_matmul(Ar.permute(0, 1, 3, 2), -Ai.permute(0, 1, 3, 2), Ar, Ai)
            Ar_inv, Ai_inv = Th_inv(Tempr, Tempi)
            return Th_comp_matmul(Ar_inv, Ai_inv, Ar.permute(0, 1, 3, 2), -Ai.permute(0, 1, 3, 2))
        elif Ar.shape[2] == Ar.shape[3]:
            return Th_inv(Ar, Ai)
    else:
        raise Exception('5-D is not defined for Th_pinv.')


In [4]:
!pip install neptune-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.1/502.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.9 MB/s eta 0:00:00
  Created wheel for bravado-core: filename=bravado_core-6.1.1-py2.py3-none-any.whl size=67672 sha256=086cb4a5766937f9b18e9438d8c8831edecb27849c451a7a8f930198afcb1e44
  Stored in directory: /root/.cache/pip/wheels/42/35/4a/44e

In [5]:
import torch.utils.data as data
from termcolor import colored
import torch.nn.functional as F
import torch
from numpy import genfromtxt
import numpy as np
# from utils_math import Th_comp_matmul, Th_inv, Th_pinv
import neptune
import re
import torch.nn as nn
import time

# Database ####################################################################################################################
class Data_Reader(data.Dataset):
    def __init__(self, filename, Us, Mr, Nrf, K, Noise_pwr):

        print(colored('You select core dataset', 'cyan'))
        print(colored(filename, 'yellow'), 'is loading ... ')
        np_data = np.load(filename)

        self.channelR = np_data['channel'].real.astype(float)
        self.channelI = np_data['channel'].imag.astype(float)

        self.RSSI_N = np_data['RSSI_N'].real.astype(float)

        self.UR = np_data['U'].real.astype(float)
        self.UI = np_data['U'].imag.astype(float)

        self.AR = np_data['A'].real.astype(float)
        self.AI = np_data['A'].imag.astype(float)

        self.WR = np_data['W'].real.astype(float)
        self.WI = np_data['W'].imag.astype(float)

        self.Noise_pwr = Noise_pwr

        self.n_samples = self.channelR.shape[0]

    def __len__(self):
        return self.n_samples

    def uniq_clas(self):
        codes = np.load('Codebook_ij.npz')['codebook']
        NO_Class = len(codes)
        print(colored("The number of Unique AP in I1: ", "green"), NO_Class)
        return NO_Class

    def rate_calculator_3d_np(self, FDP, channel):  #      FDP = (i, Nt, Nu, 1)         H = (i, Nu, 1, Nt)
        W = np.einsum('nij,njk->nik', np.conj(channel), FDP)
        diag_W = np.diagonal(np.abs(W) ** 2, axis1=1, axis2=2)
        SINR = diag_W / (np.sum(np.abs(W) ** 2, 2) - diag_W + self.Noise_pwr)
        userRates = np.log2(1 + SINR)
        sumRate = userRates.sum(1)
        return sumRate

    def optimum_HBF(self):
        A = self.AR + 1j*self.AI
        W = self.WR + 1j*self.WI
        channel = self.channelR + 1j*self.channelI
        FDP_AW = np.einsum('nij,njk->nik', A, W)
        FDP_AW = FDP_AW / np.linalg.norm(FDP_AW, axis=(1,2), keepdims=True)
        sr_HBF = Data_Reader.rate_calculator_3d_np(self, FDP_AW, channel)
        return sr_HBF.mean()

    def optimum_FDP(self):
        U = self.UR + 1j*self.UI
        channel = self.channelR + 1j*self.channelI
        U = U / np.linalg.norm(U, axis=(1,2), keepdims=True)
        sr_FDP = Data_Reader.rate_calculator_3d_np(self, U, channel)
        return sr_FDP.mean()

    def __getitem__(self, index):
        return torch.Tensor(self.channelR[index]), torch.Tensor(self.channelI[index]), torch.Tensor(self.RSSI_N[index])

# readme reader for HBF initial parameters ####################################################################################
def md_reader(DB_name):
    md = genfromtxt('DATASET.md', delimiter='\n', dtype='str')
    Us = int(re.findall(r'\d+', md[1])[0])
    Mr = int(re.findall(r'\d+', md[2])[0])
    Nrf = int(re.findall(r'\d+', md[3])[0])
    Ass_n = int(re.findall(r'\d+', md[4])[0])
    Noise_pwr = float(''.join(('1e-', str(int(int(re.findall(r'\d+', md[6])[0]) / 10)))))
    return Us, Mr, Nrf, Ass_n, Noise_pwr

class Initialization_Model_Params(object):
    def __init__(self,
                 DB_name,
                 Us,
                 Mr,
                 Nrf,
                 K,
                 K_limited,
                 Noise_pwr,
                 device,
                 device_ids
                 ):
        self.DB_name = DB_name
        self.Us = Us
        self.Mr = Mr
        self.Nrf = Nrf
        self.K = K
        self.K_limited = K_limited
        self.Noise_pwr = Noise_pwr
        self.device = device
        self.dev_id = device_ids

    def Data_Load(self):
        DataBase = Data_Reader(''.join(('DataBase_', self.DB_name, '.npz')),
                               self.Us, self.Mr, self.Nrf, self.K, self.Noise_pwr)
        uniq_dis_label = DataBase.uniq_clas()
        sr_HBF, sr_FDP = DataBase.optimum_HBF(), DataBase.optimum_FDP()
        return DataBase, uniq_dis_label, sr_HBF, sr_FDP

    def Code_Read(self):
        codes = np.load('Codebook_ij.npz')['codebook']
        # codes = genfromtxt('Codebook_ij.csv', delimiter=',', dtype='complex', skip_header=0)
        label = np.arange(len(codes))
        self.n_output_clas = len(codes)
        print(colored("The length of the codebook: ", "green"), len(codes))
        Codes_idx = np.concatenate((label[:, np.newaxis], codes), axis=1)
        codeword_C = {}
        index_C = []
        for i in range(len(codes)):
            index_C = Codes_idx[i, 0].real.astype(int)
            icode_C = Codes_idx[i, 1:]
            codeword_C[index_C] = icode_C

        # torch tensor of codes
        codesr = torch.from_numpy(codes.real).type(torch.float)
        codesi = torch.from_numpy(codes.imag).type(torch.float)
        return codeword_C, len(codes), codesr, codesi

class Loss_FDP_Rate_Based(torch.nn.Module):
    def __init__(self, Us, Mr, Nrf, Noise_pwr):
        super(Loss_FDP_Rate_Based, self).__init__()
        self.Us = Us
        self.Mr = Mr
        self.Nrf = Nrf
        self.noise_power = Noise_pwr

    def rate_calculator(self, u_re, u_im, channelr, channeli):
        Wr, Wi = Th_comp_matmul(channelr, -channeli, u_re, u_im)
        W = Wr**2 + Wi**2
        diag_W = torch.diagonal(W, dim1=1, dim2=2)

        SINR = diag_W / (torch.sum(W, 2) - diag_W + self.noise_power)
        userRates = torch.log2(1 + SINR)
        sumRate = userRates.sum(1)

        return sumRate

    def forward(self, outr, outi, channelr, channeli):
        outr = outr.view(-1, self.Us, self.Mr).permute(0, 2, 1)
        outi = outi.view(-1, self.Us, self.Mr).permute(0, 2, 1)

        # power normalization over all antennas
        temp_pre = torch.sqrt(torch.sum(outr.flatten(1) ** 2 + outi.flatten(1) ** 2, dim=1))
        outr = (outr.flatten(1) / temp_pre.unsqueeze(1)).view(outr.shape)
        outi = (outi.flatten(1) / temp_pre.unsqueeze(1)).view(outi.shape)

        sum_rate = Loss_FDP_Rate_Based.rate_calculator(self, outr, outi, channelr, channeli)
        return -sum_rate.mean()

    def evaluate_rate(self, outr, outi, channelr, channeli):
        outr = outr.view(-1, self.Us, self.Mr).permute(0, 2, 1)
        outi = outi.view(-1, self.Us, self.Mr).permute(0, 2, 1)

        # power normalization over all antennas
        temp_pre = torch.sqrt(torch.sum(outr.flatten(1) ** 2 + outi.flatten(1) ** 2, dim=1))
        outr = (outr.flatten(1) / temp_pre.unsqueeze(1)).view(outr.shape)
        outi = (outi.flatten(1) / temp_pre.unsqueeze(1)).view(outi.shape)

        sum_rate = Loss_FDP_Rate_Based.rate_calculator(self, outr, outi, channelr, channeli)
        return sum_rate.mean()

class Loss_HBF_Rate_Based_4D(torch.nn.Module):
    def __init__(self, Us, Mr, Nrf, Noise_pwr):
        super(Loss_HBF_Rate_Based_4D, self).__init__()
        self.Us = Us
        self.Mr = Mr
        self.Nrf = Nrf
        self.noise_power = Noise_pwr

    def rate_calculator_4d(self, u_re, u_im, channelr, channeli):
        Wr, Wi = Th_comp_matmul(channelr, -channeli, u_re, u_im)
        W = Wr**2 + Wi**2
        diag_W = torch.diagonal(W, dim1=2, dim2=3)
        SINR = diag_W / (torch.sum(W, 3) - diag_W + self.noise_power)
        userRates = torch.log2(1 + SINR)
        sumRate = userRates.sum(2)
        return sumRate

    def forward(self, Wr, Wi, channelr, channeli, Ar, Ai):
        HBF_prer, HBF_prei = Th_comp_matmul(Ar.view(-1, len(channelr), self.Nrf, self.Mr).permute(0, 1, 3, 2),
                                            Ai.view(-1, len(channelr), self.Nrf, self.Mr).permute(0, 1, 3, 2), Wr, Wi)

        # power normalization over all antennas
        temp_pre = torch.sqrt(torch.sum(HBF_prer.flatten(2) ** 2 + HBF_prei.flatten(2) ** 2, dim=2))
        HBF_prer = (HBF_prer.flatten(2) / temp_pre.unsqueeze(2)).view(HBF_prer.shape)
        HBF_prei = (HBF_prei.flatten(2) / temp_pre.unsqueeze(2)).view(HBF_prei.shape)

        sum_rate = Loss_HBF_Rate_Based_4D.rate_calculator_4d(self, HBF_prer, HBF_prei, channelr, channeli)
        return sum_rate.T

    def evaluate_rate(self, Wr, Wi, channelr, channeli, Ar, Ai):
        HBF_prer, HBF_prei = Th_comp_matmul(Ar.view(-1, self.Nrf, self.Mr).permute(0, 2, 1),
            Ai.view(-1, self.Nrf, self.Mr).permute(0, 2, 1), Wr.permute(0, 2, 1), Wi.permute(0, 2, 1))

        # power normalization over all antennas
        temp_pre = torch.sqrt(torch.sum(HBF_prer.flatten(1) ** 2 + HBF_prei.flatten(1) ** 2, dim=1))
        HBF_prer = (HBF_prer.flatten(1) / temp_pre.unsqueeze(1)).view(HBF_prer.shape)
        HBF_prei = (HBF_prei.flatten(1) / temp_pre.unsqueeze(1)).view(HBF_prei.shape)

        sum_rate = Loss_FDP_Rate_Based.rate_calculator(self, HBF_prer, HBF_prei, channelr, channeli)

        return sum_rate.mean()

def FLP_loss(x, y):
    log_prob = - 1.0 * F.softmax(x, 1)
    temp = log_prob * y
    cel = temp.sum(dim=1)
    cel = cel.mean()
    return cel


[neptune] [warning] NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/


In [17]:
!export CUDA_VISIBLE_DEVICES=1

In [27]:
"""
Source code for paper "Unsupervised Deep Learning for Massive MIMO Hybrid Beamforming". https://arxiv.org/abs/2007.00038.
Python codes prepared by Hamed Hojatian, 2020.
E-mail me for questions via: hamed.hojatian.ca@gmail.com.

There is an option to select "HBF-Net" or "AFP-Net".

Feel free to use this code as a starting point for your own research project.
If you do, we kindly ask that you cite the above paper.
"""

import numpy as np
import os
import csv
import torch.nn as nn
import torch.nn.functional as F
import math
from numpy import genfromtxt
import torch as th
# from networks_activation import Networks_activations
# from utils import md_reader, Initialization_Model_Params, Loss_FDP_Rate_Based, Loss_HBF_Rate_Based_4D, FLP_loss
# from utils_math import Th_pinv, Th_comp_matmul, Th_inv
from termcolor import colored
from torch.optim.lr_scheduler import ReduceLROnPlateau

###############################################################################
# Directory file
###############################################################################
DB_name = 'dataSet64x8x4_130dB_0129201820'

###############################################################################
# Processor selection GPU if available (using GPU is highly recommended)
###############################################################################
# device = th.device("cuda:2" if th.cuda.is_available() else "cpu")
device = th.device("cuda:0" if th.cuda.is_available() else "cpu")
device_ids = [0]
# device_id= [0]
print("Is Cuda available? ", colored('True', 'green')
    if th.cuda.is_available() else colored('False', 'red'))
print("Which devide?", colored(device, 'cyan'))

###############################################################################
# Setup Parameters
###############################################################################

# Beamforming approach  AFP_Net, HBF_NET   ####################################
BF_approach = 'HBF_Net'

###############################################################################
# Beamfroming system model and DNN Parameters
###############################################################################
import os

current_working_directory = os.getcwd()
Us, Mr, Nrf, K, Noise_pwr = md_reader(DB_name)                # Number of users, antenna, K, RF chains and noise power
K_limited = K                                                 # Number of SS as RSSI
batch_size = 500                                              # Batch size
epoch_size = 1000                                             # Number of training epoches
lr = 0.001                                                    # Learning rate
wd = 1e-6                                                     # Weight decay
n_input = Us * K_limited                                      # Input dimensions
n_hidden = 1024                                               # Size of FCL layers
out_channel = 16                                              # Size of CL channels
kernel_s = 3                                                  # Size of Kernels in CL
padding = 1                                                   # Size of padding in CL
p_dropout = 0.05                                              # Probability of dropout

if BF_approach == 'HBF_Net':
    n_output_reg = Us * Nrf
elif BF_approach == 'AFP_Net':
    n_output_reg = Us * Mr
else:
    raise Exception('BF_approach value is wrong !!')
###############################################################################
# Main Menu of configuration
###############################################################################
Main_Menu = Initialization_Model_Params(DB_name,
                                        Us,
                                        Mr,
                                        Nrf,
                                        K,
                                        K_limited,
                                        Noise_pwr,
                                        device,
                                        device_ids)

###############################################################################
# Reading Database
###############################################################################
DataBase, uniq_dis_label, sr_HBF, sr_FDP = Main_Menu.Data_Load()

###############################################################################
# Codeword dictionary
###############################################################################
codeword_C, n_output_clas, codesr, codesi = Main_Menu.Code_Read()
codesr = codesr.to(device)
codesi = codesi.to(device)
###############################################################################
# Training-set and test-set generation
###############################################################################
train_size = int(0.85 * len(DataBase))
test_size = len(DataBase) - train_size
train_dataset, test_dataset = th.utils.data.random_split(DataBase, [train_size, test_size])

print(colored('The size of training set is ', 'yellow'), len(train_dataset))
print(colored('The size of Test set is ', 'yellow'), len(test_dataset))

###############################################################################
# Dataloaders
###############################################################################
my_dataloader = th.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
my_testloader = th.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

###############################################################################
# DNN architecture parameters
###############################################################################
Networks_Main_Menu = Networks_activations(Us,
                                        Mr,
                                        Nrf,
                                        K,
                                        K_limited,
                                        Noise_pwr,
                                        device,
                                        device_ids,
                                        n_input,
                                        n_hidden,
                                        n_output_reg,
                                        n_output_clas,
                                        p_dropout,
                                        out_channel,
                                        kernel_s,
                                        padding)

Model_m_task = Networks_Main_Menu.Network_m_Task()

###############################################################################
# DNN OPTIMIZER
###############################################################################
optimizer_m_task = th.optim.Adam(Model_m_task.parameters(), lr=lr, weight_decay=wd)

###############################################################################
# scheduler lr
###############################################################################
scheduler_MT = ReduceLROnPlateau(optimizer_m_task, mode='max', factor=0.1, patience=5, verbose=True)

###############################################################################
# Main training loop
###############################################################################
if BF_approach == 'AFP_Net':
    # initialing the loss function
    criterium_clas_4d = Loss_HBF_Rate_Based_4D(Us, Mr, Nrf, Noise_pwr).to(device)
    criterium_reg = Loss_FDP_Rate_Based(Us, Mr, Nrf, Noise_pwr).to(device)
    for i in range(1, epoch_size):   # Main traning loop
        for k, (channelR, channelI, RSSI) in enumerate(my_dataloader):  # Loading data from data loader

            # Input data dimension check (CNN)
            Inputs_MT = Networks_Main_Menu.Inp_MT(RSSI)

            # Loading the CSI (real and imaginary)
            channelR = channelR.view(-1, Us, Mr).to(device)
            channelI = channelI.view(-1, Us, Mr).to(device)

            # Set gradient to 0.
            optimizer_m_task.zero_grad()

            # Feed forward multi-tasking DNN
            Model_m_task.train()
            out1_reg, out2_reg, out_clas = Model_m_task(Inputs_MT)

            # Computing loss for FDP in AFP-Net eq(27) in the paper
            loss_reg = criterium_reg(out1_reg, out2_reg, channelR, channelI)

            # computing the loss fucntion for HBF using eq(20)
            xx_pr, xx_pi = Th_pinv(th.unsqueeze(codesr.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).view(-1, len(RSSI), Nrf, Mr).to(device),
                th.unsqueeze(codesi.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).view(-1, len(RSSI), Nrf, Mr).to(device))
            w_outr, w_outi = Th_comp_matmul(out1_reg.view(-1, Us, Mr), out2_reg.view(-1, Us, Mr), xx_pr, xx_pi)

            HBF_all_4d = criterium_clas_4d(w_outr.permute(0, 1, 3, 2), w_outi.permute(0, 1, 3, 2), channelR, channelI,
                th.unsqueeze(codesr.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).to(device), th.unsqueeze(codesi.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).to(device))

            loss_clas = FLP_loss(out_clas, HBF_all_4d)

            # total loss fucntion eq(29)
            loss = loss_clas + loss_reg

            # Gradient calculation.
            loss_clas.backward(retain_graph=True)
            loss_reg.backward(retain_graph=True)
            loss.backward()

            # Model weight modification based on the optimizer.
            optimizer_m_task.step()

            # iterate through test dataset
            if k == 0 or i % epoch_size == 0:
                del loss
                # No gardient in test mode
                with th.no_grad():
                    R_predicted_HBF = []
                    R_optimum_HBF = []
                    R_optimum_FDP = []
                    R_predicted_FDP = []
                    Rate_Ratio_HBF = []
                    Rate_Ratio_FDP = []
                    for (tchannelR, tchannelI, tRSSI) in my_testloader:

                        # Input data dimension check (CNN)
                        testInputs_Reg = Networks_Main_Menu.Inp_MT(tRSSI)

                        # Loading the near-optimal digital precoder, CSI (real and imaginary)
                        T_channelR = tchannelR.reshape(-1, Us, Mr).to(device)
                        T_channelI = tchannelI.reshape(-1, Us, Mr).to(device)

                        # Forward pass test mode DNN
                        Model_m_task.eval()
                        pred1_reg, pred2_reg, pred_class = Model_m_task(testInputs_Reg)

                        # find the maximum probability as predication of classification
                        _, predicted = th.max(F.softmax(pred_class, 1), 1)

                        # mapping in the codebook to find the corresponding analog precoder
                        An_Predr = codesr[predicted, :].to(device)
                        An_Predi = codesi[predicted, :].to(device)

                        # finding digital precoder using eq(20)
                        x_pr, x_pi = Th_pinv(An_Predr.view(-1, Nrf, Mr), An_Predi.view(-1, Nrf, Mr))
                        w_prer, w_prei = Th_comp_matmul(pred1_reg.view(-1, Us, Mr), pred2_reg.view(-1, Us, Mr), x_pr, x_pi)

                        # rate calculation
                        # DNN HBF
                        R_predicted_HBF.append(criterium_clas_4d.evaluate_rate(w_prer, w_prei, T_channelR, T_channelI, An_Predr, An_Predi))
                        # DNN FDP
                        R_predicted_FDP.append(criterium_reg.evaluate_rate(pred1_reg, pred2_reg, T_channelR, T_channelI))

                # Average over all mini-batches
                RATE_Predicted_HBF = sum(R_predicted_HBF) / len(R_predicted_HBF)
                RATE_Predicted_FDP = sum(R_predicted_FDP) / len(R_predicted_FDP)
                RATE_Ratie_HBF = 100 * RATE_Predicted_HBF / sr_HBF
                RATE_Ratie_FDP = 100 * RATE_Predicted_FDP / sr_FDP

                scheduler_MT.step(RATE_Predicted_HBF)

                print('Iter:==>{:3d} Loss_FDP:{:.3f} Loss_Class:{:.3f} Rate_opt_HBF:{:.2f} Rate_opt_FDP:{:.2f} Rate_pre_HBF:{:.2f} Rate_pre_FDP:{:.2f} Ratio_HBF:{:.2f}% Ratio_FDP:{:.2f}%'.
                    format(i, loss_reg, loss_clas, sr_HBF, sr_FDP, RATE_Predicted_HBF, RATE_Predicted_FDP, RATE_Ratie_HBF, RATE_Ratie_FDP))

elif BF_approach == 'HBF_Net':
    # initialing the loss function
    criterium_clas_4d = Loss_HBF_Rate_Based_4D(Us, Mr, Nrf, Noise_pwr).to(device)
    for i in range(1, epoch_size):
        for k, (channelR, channelI, RSSI) in enumerate(my_dataloader):

            # Input data dimension check (CNN)
            Inputs_Reg = Networks_Main_Menu.Inp_MT(RSSI)

            # Loading the CSI (real and imaginary)
            channelR = channelR.view(-1, Us, Mr).to(device)
            channelI = channelI.view(-1, Us, Mr).to(device)

            # Set gradient to 0.
            optimizer_m_task.zero_grad()

            # Feed forward multi-tasking DNN
            Model_m_task.train()
            out1_reg, out2_reg, out_clas = Model_m_task(Inputs_Reg)

            # computing the loss fucntion for HBF using eq(25)
            w_outr, w_outi = out1_reg.view(-1, Us, Nrf), out2_reg.view(-1, Us, Nrf)
            HBF_all_4d = criterium_clas_4d(w_outr.permute(0, 2, 1), w_outi.permute(0, 2, 1), channelR, channelI,
                th.unsqueeze(codesr.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).to(device),
                th.unsqueeze(codesi.unsqueeze(1), 2).repeat(1, len(RSSI), 1, 1).to(device))
            loss_clas = FLP_loss(out_clas, HBF_all_4d)

            # Gradient calculation.
            loss_clas.backward()

            # Model weight modification based on the optimizer.
            optimizer_m_task.step()

            # iterate through test dataset
            if k == 0 or i % epoch_size == 0:
                R_predicted_HBF = []
                R_optimum_HBF = []
                Rate_Ratio_HBF = []
                with th.no_grad():
                    for (tchannelR, tchannelI, tRSSI) in my_testloader:

                        # Input data dimension check (CNN)
                        testInputs_Reg = Networks_Main_Menu.Inp_MT(tRSSI)

                        # Loading the near-optimal digital precoder, CSI (real and imaginary)
                        T_channelR = tchannelR.reshape(-1, Us, Mr).to(device)
                        T_channelI = tchannelI.reshape(-1, Us, Mr).to(device)

                        # Forward pass reg
                        Model_m_task.eval()
                        pred1_reg, pred2_reg, pred_class = Model_m_task(testInputs_Reg)

                        # find the maximum probability as predication of classification
                        _, predicted = th.max(F.softmax(pred_class, 1), 1)

                        # mapping in the codebook to find the corresponding analog precoder
                        An_Predr = codesr[predicted, :]
                        An_Predi = codesi[predicted, :]
                        w_prer, w_prei = pred1_reg.view(-1, Us, Nrf), pred2_reg.view(-1, Us, Nrf)

                        # rate calculation
                        # DNN HBF
                        R_predicted_HBF.append(criterium_clas_4d.evaluate_rate(w_prer, w_prei, T_channelR, T_channelI, An_Predr, An_Predi))

                # Average over all mini-batches
                RATE_Predicted_HBF = sum(R_predicted_HBF) / len(R_predicted_HBF)
                RATE_Ratie_HBF = 100 * RATE_Predicted_HBF / sr_HBF

                scheduler_MT.step(RATE_Predicted_HBF)

                print('Iter:==>{:3d} Loss_Class:{:.3f} Rate_opt_HBF:{:.2f} Rate_pre_HBF:{:.2f} Ratio_HBF:{:.2f}%'.
                    format(i, loss_clas, sr_HBF, RATE_Predicted_HBF, RATE_Ratie_HBF))

else:
    raise Exception('BF_approach is wrong !!')


Is Cuda available?  True
Which devide? cuda:0
You select core dataset
DataBase_dataSet64x8x4_130dB_0129201820.npz is loading ... 
The number of Unique AP in I1:  5
The length of the codebook:  5
The size of training set is  8500
The size of Test set is  1500


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Iter:==>  1 Loss_Class:-0.343 Rate_opt_HBF:7.87 Rate_pre_HBF:0.73 Ratio_HBF:9.32%
Iter:==>  2 Loss_Class:-3.636 Rate_opt_HBF:7.87 Rate_pre_HBF:3.43 Ratio_HBF:43.61%
Iter:==>  3 Loss_Class:-4.013 Rate_opt_HBF:7.87 Rate_pre_HBF:4.20 Ratio_HBF:53.30%
Iter:==>  4 Loss_Class:-4.135 Rate_opt_HBF:7.87 Rate_pre_HBF:4.46 Ratio_HBF:56.70%
Iter:==>  5 Loss_Class:-4.226 Rate_opt_HBF:7.87 Rate_pre_HBF:4.57 Ratio_HBF:58.09%
Iter:==>  6 Loss_Class:-4.193 Rate_opt_HBF:7.87 Rate_pre_HBF:4.63 Ratio_HBF:58.83%
Iter:==>  7 Loss_Class:-4.298 Rate_opt_HBF:7.87 Rate_pre_HBF:4.68 Ratio_HBF:59.44%
Iter:==>  8 Loss_Class:-4.362 Rate_opt_HBF:7.87 Rate_pre_HBF:4.69 Ratio_HBF:59.54%
Iter:==>  9 Loss_Class:-4.470 Rate_opt_HBF:7.87 Rate_pre_HBF:4.71 Ratio_HBF:59.87%
Iter:==> 10 Loss_Class:-4.445 Rate_opt_HBF:7.87 Rate_pre_HBF:4.73 Ratio_HBF:60.09%
Iter:==> 11 Loss_Class:-4.420 Rate_opt_HBF:7.87 Rate_pre_HBF:4.74 Ratio_HBF:60.23%
Iter:==> 12 Loss_Class:-4.577 Rate_opt_HBF:7.87 Rate_pre_HBF:4.76 Ratio_HBF:60.41%
Iter:

In [23]:
torch.cuda.is_available()


True

In [24]:
torch.cuda.device_count()

1

In [25]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available CUDA devices
    num_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_devices)

    # Iterate over each CUDA device and print its index
    for i in range(num_devices):
        device_name = torch.cuda.get_device_name(i)
        print(f"Device ordinal for '{device_name}': cuda:{i}")
else:
    print("CUDA is not available on this system.")


Number of CUDA devices: 1
Device ordinal for 'Tesla T4': cuda:0
